# 布尔查询与倒排索引
### 流程
- 数据预处理
- 建立到排索引
- 实现query的AND，NOT，OR逻辑
- 查询操作返回topK结果

In [13]:
import json
import nltk

### 数据预处理
- 解析json格式数据得到dict_list
- 去非关键性标点，换行符等，split
- 建立vocab（Tokenize）
- 保存结果数据
###### 注意这里的 text texts的顺序相同 vbocab 一个文档只记录一次

In [14]:
f = open('./data/tweets.txt','r+')
lines = f.readlines()
text = []
for l in lines:
    text.append(json.loads(l)['text'])

#### 去除特殊符号
- 保留完整网址

In [15]:
texts = []
symbol = [',',':','_','!','\"','*','>','<','@','~','-','(',')','%','=','\\','^'
          ,'&','|','#','$','[',']','+',':','#','|'] 
for l in text:
    for s in symbol:
        line = l.replace(s,'')
    line = line.split()
    texts.append(line)

#### 建立vocab

In [16]:
vocab = {}
for line in texts:
    line_set = set(line) ##去重
    for word in line_set:       
            if vocab.get(word) == None:
                vocab[word] = 1
            else :
                vocab[word] += 1
v_tuple = sorted(vocab.items(),key = lambda x:x[1],reverse=True) ##从大到小排序
vocab = {}
for t in v_tuple:
    vocab[t[0]] = t[1]

#### 保存

In [17]:
f = open('./data/vocab.txt','w+')
line_str = json.dumps(vocab)
f.write(line_str)
f.close()

### 建立到倒排索引

In [20]:
word2inverted_index = {} ##一个字典 存frequency 一个字典存 每个词出现的docID
keys = list(vocab.keys())
for k in keys:
    word2inverted_index[k] = []##初始化

for i,line in enumerate(texts):
    line_set = set(line) ##去重
    for word in line_set:
        word2inverted_index[word].append(i)

### 实现query的AND，NOT，OR逻辑

In [28]:
def AND_op(l1,l2):
    ##两个有序链表 返回 共同的 docID（交集）有序
    result = []
    i = 0
    j = 0
    while(i<len(l1) and j < len(l2)):
        if l1[i] == l2[j]:
            result.append(l1[i])
            i += 1
            j += 1
        else:
            if l1[i] < l2[j]:
                i += 1
            else:
                j += 1
    return result

In [36]:
def OR_op(l1,l2):
    ##两个有序链表 返回并集 docID 并且去重 有序
    result = []
    i = 0
    j = 0
    while(i<len(l1) and j < len(l2)):
        if l1[i] == l2[j]:
            result.append(l1[i])
            
            i += 1
            j += 1
        else:
            if l1[i] < l2[j]:
                result.append(l1[i])
                i += 1
            else:
                result.append(l2[j])
                j += 1
    result.extend(l1[i:])
    result.extend(l2[j:])
    ##注意这里注意剩余
    return result

In [56]:
def NOT_op(l):
    ##一个有序链表 返回补集 有序
    result = []
    j = 0
    for i in range(l[-1]):
        if l[j] == i:
            j += 1
        else:
            result.append(i)
    ##补全
    for i in range(l[j]+1,len(texts)):
        result.append(i)
    return result

### 布尔查询
#### 支持
- NOT否定运算
- AND交集运算
- OR并集运算
- 上述运算具有优先级
- ()运算，并且支持嵌套

#### 字符串解析与计算
- 中缀表达式变成后缀表达式
- 后缀表达式的计算

In [117]:
ops_rule = {
    '+': 1,
    '*': 2,
} ##优先级定义

def middle_to_after(ss):
    expression = []
    ops = []
    for item in ss:
        if item in ['+', '*']:
            while len(ops) >= 0:
                if len(ops) == 0:
                    ops.append(item)
                    break
                op = ops.pop()
                if op == '(' or ops_rule[item] > ops_rule[op]:
                    ops.append(op)
                    ops.append(item)
                    break
                else:
                    expression.append(op)
        elif item == '(':
            ops.append(item)
        elif item == ')':
            while len(ops) > 0:
                op = ops.pop()
                if op == '(':
                    break
                else:
                    expression.append(op)
        else:
            expression.append(item)

    while len(ops) > 0:
        expression.append(ops.pop())

    return expression

def expression_to_value(expression):
    stack_value = []
    for item in expression:
        if item in ['+', '*']:
            n2 = stack_value.pop()
            n1 = stack_value.pop()
            result = cal(n1, n2, item)
            stack_value.append(result)
        else:
            stack_value.append(item)
    return stack_value[0]
 
def cal(n1, n2, op):
    if op == '+':
        return OR_op(n1,n2)
    if op == '*':
        return AND_op(n1,n2)

#### 先处理NOT再进行其他操作的运算
- 支持括号运算

In [118]:
def boolean_search(Q):
    ##输入特定的query进行解析
    s_line = Q.split()
    l = []
    op = []
    ##数字和list
    i = 0
    ##解决 not 问题
    while i < len(s_line):
        s = s_line[i]
        if s == 'NOT':
            op.append(NOT_op(word2inverted_index[s_line[i+1]]))
            i += 1
        else:
            if s == 'AND':
                op.append('*') ##优先级高
            elif s == 'OR':
                op.append('+') ##优先级低
            elif s == '(':
                op.append('(') 
            elif s == ')':
                op.append(')') 
            else:
                op.append(word2inverted_index[s])
        i += 1
    return expression_to_value(middle_to_after(op))

#### 返回topk
- 这里k取20

In [155]:
def top_k_search(Q):
    k = 20
    return [texts[i] for i in boolean_search(Q)[:20]] ##这里自动返回 前20个 不足 不会补全

#### 这个方法会让打印出来的字符改变颜色强调
- 关键词变成红色起到调的作用

In [160]:
def inred( s ):
    return"%s[31;2m%s%s[0m"%(chr(27), s, chr(27))
def print_line_with_important(line , Q):
    Q = Q.replace('NOT','')
    Q = Q.replace('AND','')
    Q = Q.replace('OR','')
    Q = Q.replace('(','')
    Q = Q.replace(')','')
    #只剩关键词语
    
    s_line = Q.split()
    
    to_be_print = ''
    for l in line:
        if l in s_line:
            to_be_print += inred(l) +' ' ##这里强调
        else :
            to_be_print += l + ' '
    print(to_be_print)
def print_top_k(Q):
    for line in top_k_search(Q):
        print_line_with_important(line,Q)

### 测试查询结果
- 效果感觉非常不错

In [169]:
print_top_k('Apple')

Tim Cook steps in for Steve Jobs at Apple - San Francisco Chronicle http://bit.ly/dYzbN4 
IJSMblog Week: Does Apple have a future without Steve Jobs? http://bit.ly/hT82a5 
What Would an Apple Without Steve Jobs Be Like? – Newsweek: CTV.ca What Would an Apple… http://goo.gl/fb/ooiEb 
What Would an Apple Without Steve Jobs Be Like? - Newsweek http://goo.gl/fb/iikvB 
Blog Post:8 Potential Replacements for Steve Jobs at Apple http://tumblr.com/x231bxmp87 via @fastcompany 
-&gt;@TechCrunch: DLD11: James Murdoch On The Daily, Paywalls, Google And Apple http://bit.ly/g53dEG 
Kumaran : DLD11: James Murdoch On The Daily, Paywalls, Google And Apple http://zah.cc/rx4 
DLD11: James Murdoch On The Daily, Paywalls, Google And Apple http://bit.ly/gRTTzw 
DLD11: James Murdoch On The Daily, Paywalls, Google And Apple http://bit.ly/gRTTzw #tech 
DLD11: James Murdoch On The Daily, Paywalls, Google And Apple http://goo.gl/fb/UZPsG 
Apple Leads Smartphone Race, while Android Attracts Most Recent Customers 

In [170]:
print_top_k('A AND NOT Apple')

A Shredded BBQ Chicken Sliders with Creamy Cole Slaw #recipe was just added: http://bit.ly/gPMYh1 
Australia braces for more floods: A giant inland sea of floodwater will spread across the Australian state of Vi... http://bbc.in/i1wSfM 
40st McDonalds A Effin Zoo!! Lol 
Credit Card Consolidation Help – How To Use A Debt Settlement To Consolidate Credit Bills http://tinyurl.com/4h8vajn 
@lukewilliamss Brain fluid buildup delays rehab for Congresswoman Giffords: A buildup of fluid in... http://bit.ly/gzZg9Z #lukewilliamss 
www.webmarkloans.com Consult Your Debt Consolidation Company For Financial Issues: A debt consolidation company ... http://bit.ly/gYFWIs 
Dutchman charged in Yeates murder hunt (AFP) - AFP - A 32-year-old Dutchman was charged Saturday with the murde... http://ow.ly/1aZqxx 
50 Cent at Sundance: A work in progess http://bit.ly/fjq9uC 
Aristide Must Come Back Before A Republican President Gets Elected In US: Well said TIBA, we are paying the pri... http://bit.ly/hT4Cde 
W

In [172]:
print_top_k('A AND Apple')

Apple nkor"@OtunbaSula: RIM bawo? : “@MOHYEES: A prolific writer. He controlled pen and paper. RIM Chinua Achebe. We will surely miss you”" 


In [176]:
print_top_k('( A OR Will ) AND Apple')

eMarketer: Apple Will Soon Lead The US Smartphone Market – But Not For Long: According to eMarketer, Apple is t... http://bit.ly/gOqCej 
#tech eMarketer: Apple Will Soon Lead The US Smartphone Market – But Not For Long http://bit.ly/euA43t 
eMarketer: Apple Will Soon Lead The US Smartphone Market – But Not For Long: According to eMarketer, Apple is t... http://bit.ly/euA43t 
eMarketer: Apple Will Soon Lead The US Smartphone Market – But Not For Long: According to eMarketer, Apple is t... http://bit.ly/gOqCej 
eMarketer: Apple Will Soon Lead The US Smartphone Market – But Not For Long: According to eMarketer, Apple is t... http://bit.ly/fpNBr8 
eMarketer: Apple Will Soon Lead The US Smartphone Market – But Not For Long: According to eMarketer, Apple is t... http://bit.ly/f5MYLk 
Apple Will Soon Lead The US Smartphone Market  But Not For Long - by @bonairedotcom 27/01/2011 @ 19:12: Acco... http://bit.ly/hkgGSn 
DigiC.Live Murdoch Hopes Apple Will Lower Its Share Of The Daily Take (Staci